In [2]:
import sys
sys.path.append('../../')

In [3]:
from keras.models import load_model
model_path = '../../results/ranking_models_from_scratch/q1_model_training/hypermodel_tuning/BayesianOptimization_Q1_Datasets_rs_noseed/BayesianOptimization_2310071922/Model_1_Ranking_VGG19/Meta_Ranking_Network.keras'
meta_model = load_model(model_path)

# get the ranking network
ranking_network = meta_model.get_layer('Ranking_Network')
ranking_network.summary()

# compile the ranking network
compile_args = {'optimizer': meta_model.optimizer, 'loss': meta_model.loss, 'metrics': meta_model.metrics}

Model: "Ranking_Network"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_Image (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 Flatten (Flatten)           (None, 25088)             0         
                                                                 
 Dense_1 (Dense)             (None, 80)                2007120   
                                                                 
 Dense_1_BN (BatchNormalizat  (None, 80)               320       
 ion)                                                            
                                                                 
 Dense_1_Drop (Dropout)      (None, 80)                0         
                                                   

In [4]:
vgg19_backbone = ranking_network.get_layer('vgg19')
vgg19_backbone.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [5]:
weights_vgg19_backbone = vgg19_backbone.get_weights()
weights_vgg19_backbone

[array([[[[ 0.3392252 ,  0.0953927 ,  0.01648431, ..., -0.11933026,
           -0.0523739 , -0.00367317],
          [ 0.46256614,  0.03267206,  0.10127774, ..., -0.07611246,
           -0.04177195,  0.03975039],
          [ 0.39201662, -0.08682787, -0.03915127, ..., -0.11490995,
           -0.04037674,  0.0453885 ]],
 
         [[ 0.33804756,  0.13398238,  0.02019014, ..., -0.13405852,
           -0.1663438 , -0.07018261],
          [ 0.41644415,  0.05210221,  0.09641077, ..., -0.10021169,
           -0.20648508, -0.05791578],
          [ 0.3752397 , -0.08092276, -0.05062415, ..., -0.12534584,
           -0.19242425, -0.02026838]],
 
         [[-0.04721516,  0.06505761, -0.07792876, ..., -0.12996043,
           -0.3004481 , -0.28328386],
          [-0.0502468 , -0.00739582, -0.02409653, ..., -0.09414877,
           -0.39338574, -0.3473438 ],
          [-0.04861257, -0.11849625, -0.14802507, ..., -0.12963572,
           -0.36045745, -0.28156137]]],
 
 
        [[[ 0.23009597,  0.1334942

In [6]:
import numpy as np
from keras import layers
weights_arrays = []
for i in range (1,11):
    # load meta model
    model_path = f'../../results/ranking_models_from_scratch/q1_model_training/hypermodel_tuning/BayesianOptimization_Q1_Datasets_rs_noseed/BayesianOptimization_2310071922/Model_{i}_Ranking_VGG19/Meta_Ranking_Network.keras'
    meta_model = load_model(model_path)
    
    # get the backbone weights
    ranking_network = meta_model.get_layer('Ranking_Network')
    vgg19_backbone = ranking_network.get_layer('vgg19')
    weights_vgg19_backbone = [layer.get_weights() for layer in vgg19_backbone.layers if len(layer.get_weights()) > 0]
    weights_arrays.append(weights_vgg19_backbone)
len(weights_arrays)

10

In [7]:
# display the weights of the each layer in VGG19 backbone
for i in range(len(weights_arrays[0])):
    print(f'Layer {i+1}: {weights_arrays[0][i]}')
# Calculate the average weights for each layer in 10 vgg19 models


Layer 1: [array([[[[ 0.3392252 ,  0.0953927 ,  0.01648431, ..., -0.11933026,
          -0.0523739 , -0.00367317],
         [ 0.46256614,  0.03267206,  0.10127774, ..., -0.07611246,
          -0.04177195,  0.03975039],
         [ 0.39201662, -0.08682787, -0.03915127, ..., -0.11490995,
          -0.04037674,  0.0453885 ]],

        [[ 0.33804756,  0.13398238,  0.02019014, ..., -0.13405852,
          -0.1663438 , -0.07018261],
         [ 0.41644415,  0.05210221,  0.09641077, ..., -0.10021169,
          -0.20648508, -0.05791578],
         [ 0.3752397 , -0.08092276, -0.05062415, ..., -0.12534584,
          -0.19242425, -0.02026838]],

        [[-0.04721516,  0.06505761, -0.07792876, ..., -0.12996043,
          -0.3004481 , -0.28328386],
         [-0.0502468 , -0.00739582, -0.02409653, ..., -0.09414877,
          -0.39338574, -0.3473438 ],
         [-0.04861257, -0.11849625, -0.14802507, ..., -0.12963572,
          -0.36045745, -0.28156137]]],


       [[[ 0.23009597,  0.13349429,  0.1203201

In [23]:
average_weights = []
for i in range(len(weights_arrays[0])):
    # Check if there are weights for this layer
    if weights_arrays[0][i]:
        # Calculate the average of weights from the three models
        avg_layer_weights = (np.array(weights_arrays[0][i]) + 
                             np.array(weights_arrays[1][i]) + 
                             np.array(weights_arrays[2][i]) + 
                             np.array(weights_arrays[3][i]) + 
                             np.array(weights_arrays[4][i]) + 
                             np.array(weights_arrays[5][i]) + 
                             np.array(weights_arrays[6][i]) + 
                             np.array(weights_arrays[7][i]) + 
                             np.array(weights_arrays[8][i]) + 
                             np.array(weights_arrays[9][i])) / 10.0
        average_weights.append(avg_layer_weights.tolist())
    else:
        # If no weights for this layer, use an empty list
        average_weights.append([])

KeyboardInterrupt: 

In [ ]:
from keras.applications import VGG19
# Create a new VGG19 model with the averaged weights
avg_vgg19_model = VGG19(weights=None, include_top=False)
for i, layer in enumerate(avg_vgg19_model.layers):
    if average_weights[i]:
        layer.set_weights(average_weights[i])